# 文本分类

实施步骤：
+ 解压缩文件，读取每个文件构建可以使用的训练集测试机
+ 训练doc2vec构建模型
+ 对每个文档运行doc2vec 转成向量
+ 构建神经网络分类算法
+ 训练分类，之后调参验证

In [ ]:
from urllib2 import urlopen, URLError, HTTPError
import os
save_path= './'
url = 'http://www.qwone.com/~jason/20Newsgroups/20news-bydate.tar.gz'
f = urlopen(url,timeout=1000)
print "downloading " + url

        # Open our local file for writing
with open(os.path.basename(url), "wb") as local_file:
    local_file.write(f.read())
    print "finish"


In [ ]:
data_dir = './'
import helper
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

#helper.download_extract(data_dir) #下载并解压文档 同时将文档数据存储到params.p中去 以便于调用

In [ ]:
import os
import sys
sentences,collect = helper.read_file() # 将所有文档读入对象并统计每一类个数

print collect
print(sentences['20news-bydate-train'][:2])
print(sentences['20news-bydate-test'][:2])


# checkpoint 2

In [ ]:
raw_data = helper.load_params() #获取数据集
#print raw_data['20news-bydate-train'][0]
#print raw_data['20news-bydate-test'][0]

# 绘制文档数量柱状图 
X_label = []
Y1 = []
Y2 = []
for i,v in collect['20news-bydate-train'].items():
    X_label.append(i)
    Y1.append(v)
for i,v in collect['20news-bydate-test'].items():   
    Y2.append(v)
X=np.arange(20)
#print X
#print Y1
plt.xticks(X, X_label)
plt.xticks(rotation=90)

plt.bar(X,Y1,width = 0.35,facecolor = 'lightskyblue',edgecolor = 'white',label= 'train')
#width:柱的宽度
plt.bar(X+0.35,Y2,width = 0.35,facecolor = 'yellowgreen',edgecolor = 'white',label= 'test')
plt.legend()

plt.show()

In [ ]:
#统计
train_no = len(raw_data['20news-bydate-train'])
test_no = len(raw_data['20news-bydate-test'])
total_news = train_no + test_no
print('训练集共有新闻：%s\n测试集共有新闻：%s\n新闻总数：%s\n '%(train_no,test_no, total_news ))
# 统计每一类
from collections import Counter
total_counts = Counter()
word_total_each = []
for type in raw_data:    
    for news in raw_data[type]:
        i = 0
        for word in news[0].split(" "):
            total_counts[word] += 1  
            i += 1
        word_total_each.append(i)
word_total_each = np.array(word_total_each)
print total_counts.most_common(100) #获取频率最高的单词
print sum(total_counts.values()) #获取单词总数
print len(total_counts) #获取单词种类
print len(word_total_each)
print word_total_each[:10]
print np.median(word_total_each),np.mean(word_total_each),np.std(word_total_each),np.amax(word_total_each),np.amin(word_total_each),np.percentile(word_total_each, 25),np.percentile(word_total_each, 75)
print total_counts.most_common()[:-100:-1] # 获取频率最低的单词

### 构建doc2vec

In [ ]:
import gensim
import random
from random import shuffle
import datetime
from gensim.models import Doc2Vec

from collections import OrderedDict
from gensim.parsing.preprocessing import STOPWORDS

第一次错误的实现代码

In [ ]:
'''
import gensim
import random
from random import shuffle
import datetime
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict

LabeledSentence = gensim.models.doc2vec.LabeledSentence
x_train,y_train = helper.labeled_data(raw_data['20news-bydate-train'])
x_test,y_test = helper.labeled_data(raw_data['20news-bydate-test'])

#将news转化为doc2vec model可以处理的对象
def labelizeDoc(doc, label,y):
    labelized = []
    for i,v in enumerate(doc):
        #str1 = ''.join(str(e) for e in y[i])
        tag = '%s_%s_%s'%(label,i,y[i])
        labelized.append(LabeledSentence(v,[tag]))
    return labelized

x_train = labelizeDoc(x_train, 'TRAIN',y_train)
x_test = labelizeDoc(x_test, 'TEST',y_test)
print y_train

%%time


size = 100
min_count = 1
window=10
negative=5
dm=0

#构建两种模型融合的向量
model_dm = gensim.models.Doc2Vec(min_count=min_count, window=window, size=size, sample=1e-3, negative=negative, workers=4)

model_dbow = gensim.models.Doc2Vec(min_count=min_count, window=window, size=size, sample=1e-3, negative=negative, dm=dm, workers=4)
#model_dm = gensim.models.Doc2Vec.load('db.pkl')
#model_dbow = gensim.models.Doc2Vec.load('dm.pkl')
listall = x_train + x_test
l = len(listall)

model_dm.build_vocab(listall)
model_dbow.build_vocab(listall)



model_dm.train(listall,total_examples=model_dm.corpus_count, epochs=20)
model_dbow.train(listall,total_examples=model_dbow.corpus_count, epochs=20)

def getVecs(model, corpus, size):
    vecs = [np.array(model.infer_vector(corpus[z].words)).reshape((1,size)) for z in range(l)]
    return np.concatenate(vecs)



train_vecs_dm = getVecs(model_dm,listall,size)
train_vecs_dbow = getVecs(model_dbow,listall,size)
model_dbow.save('db.pkl')
model_dm.save('dm.pkl')
train_vecs = np.hstack((train_vecs_dm, train_vecs_dbow))

#print listall
#pickle.dump(train_vecs, open('vec.p', 'wb'))
'''

# 正确的代码开始 一种新的方法显示文档转向量

In [ ]:
#定义函数将普通文档转换维doc2vec可以接受的 TaggedDocument类
def read_corpus(data,typename,idently):    
    i = 0
    for i, v in enumerate(data):       
        tagname = typename + '_'+str(i)+'_'+ str(idently[i])  
        #不用去除停止词 更准确
        yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(v[0]), [tagname])         
        #yield gensim.models.doc2vec.TaggedDocument([token for token in gensim.utils.simple_preprocess(v[0]) if token not in STOPWORDS], [tagname])

## 开始训练
根据官方示例 需要多次训练并且每次训练打乱原始数据

In [ ]:
# 获取训练集和测试集数据
x_train,y_train = helper.labeled_data(raw_data['20news-bydate-train'])
x_test,y_test = helper.labeled_data(raw_data['20news-bydate-test'])
#因为训练词向量 所以使用所有数据而不是仅仅测试集
#x_train += x_test
#y_train += y_test  #这里不再将测试集加入到词向量的训练中去

train_corpus = list(read_corpus(x_train, 'TRAIN',y_train))

alpha, min_alpha, passes = (0.025, 0.001, 20)
alpha_delta = (alpha - min_alpha) / passes
#最佳模型采用 PV-DBOW ，size表示最后输出维度，min_count 表示多大重复的字才会被计算到词典中，这里使用2是增加多次出现词的权重
simple_models = [    
    # PV-DM w/concatenation - window=5 (both sides) approximates paper's 10-word total window size
    #Doc2Vec(dm=1, dm_concat=1, size=100, window=5, negative=5, hs=0, min_count=2),
    # PV-DBOW 
    Doc2Vec(dm=0, size=100, negative=5, hs=0, min_count=2),
    # PV-DM w/average
    #Doc2Vec(dm=1, dm_mean=1, size=100, window=10, negative=5, hs=0, min_count=2),
]
simple_models[0].build_vocab(train_corpus) # 词典只用建立一次
models_by_name = OrderedDict((str(model), model) for model in simple_models)

print("START %s" % datetime.datetime.now())
for epoch in range(passes):
   
    shuffle(train_corpus)  # shuffling gets best results
    
    for name, train_model in models_by_name.items():
        # train
       
        train_model.alpha, train_model.min_alpha = alpha, alpha  # 随着训练不断减小alpha
        
        # 模型训练 train_model.iter 未定义时默认是5 ，所以20次迭代实际一共训练 5*20= 100 次
        train_model.train(train_corpus, total_examples=train_model.corpus_count, epochs=train_model.iter) 
    
    if ((epoch + 1) % 5) == 0 or epoch == 0: 
        print 'epoch %s finish'%epoch
    alpha -= alpha_delta
    
print("END %s" % str(datetime.datetime.now()))

#储存doc2vec的model
train_model.save('docvec')



使用训练好的模型 把备选类别中的文档转化为向量

In [ ]:
train_model = Doc2Vec.load('best')
#获取这四类新闻的文本和label
x_train,y_train = helper.choiced_data(raw_data['20news-bydate-train'])
x_test,y_test = helper.choiced_data(raw_data['20news-bydate-test'])

#将文本转化为文档向量
x_train_vec = []
x_test_vec = []
for i in x_train:
    # 不去除停止词 
    #inferred_vector = train_model.infer_vector([token for token in gensim.utils.simple_preprocess(i[0]) if token not in STOPWORDS])
    inferred_vector = train_model.infer_vector(gensim.utils.simple_preprocess(i[0])) #这里注意输入必须是数组 而不是文本
    x_train_vec.append(inferred_vector)
for j in x_test:
    #inferred_vector_test = train_model.infer_vector([token for token in gensim.utils.simple_preprocess(j[0]) if token not in STOPWORDS])
    inferred_vector_test = train_model.infer_vector(gensim.utils.simple_preprocess(j[0]))
    x_test_vec.append(inferred_vector_test)

print x_train_vec[0],y_train[0]
print x_test_vec[0],y_test[0]

# 分组向量 画图可视化

In [ ]:
# 不同的类别在图上给予不同的颜色 对四个类别的训练集做展示
from sklearn.manifold import TSNE
ts = TSNE(2)
reduced_vecs = ts.fit_transform(np.concatenate((x_train_vec[:593],x_train_vec[593:1192],x_train_vec[1192:1783],x_train_vec[1783:])))
print reduced_vecs.shape
for i in range(len(reduced_vecs)):
    if i < 593:
        color = 'b'
    elif i>=593 and i<1192:
        color = '#d02626'
    elif i>=1192 and i<1783:
        color = 'g'
    elif i>=1783:
        color = 'y'
    plt.plot(reduced_vecs[i,0],reduced_vecs[i,1],marker='o',color=color,markersize=3)

# 构建神经网络分类模型

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier

# 因为算法有设置自动停止 所以步长设置的比较小，迭代次数可以放的多
model =  MLPClassifier(hidden_layer_sizes=90,max_iter=5000,random_state=0,verbose=True,alpha=0.001,learning_rate_init=0.0001)
scaler = MinMaxScaler()
scaler.fit(x_train_vec)

# 对得到的向量进行统一的标准化处理
x_train_vec = scaler.transform(x_train_vec)
x_test_vec = scaler.transform(x_test_vec)
#print y_train
model.fit(x_train_vec,y_train)
pp = model.predict(x_test_vec)

from sklearn.metrics import classification_report,confusion_matrix

# 展示一些统计结果 验证准确度
print(confusion_matrix(y_test,pp))
print(classification_report(y_test,pp))


score = model.score(x_test_vec, y_test)
print score

# 网格搜索获取最佳模型

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'hidden_layer_sizes':[90,70,40], 'alpha':[1, 10, 0.01, 0.001],'learning_rate_init':[0.001,0.01,0.0001]}
model =  MLPClassifier(max_iter=1000,random_state=0)
clf = GridSearchCV(model, parameters)
clf.fit(x_train_vec,y_train)
print clf.cv_results_
print clf.best_params_
print clf.best_score_

# 使用朴素贝叶斯分类方法

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB

# 因为算法有设置自动停止 所以步长设置的比较小，迭代次数可以放的多
model =  MultinomialNB(alpha=1, fit_prior=True, class_prior=None)
scaler = MinMaxScaler()
scaler.fit(x_train_vec)

# 对得到的向量进行统一的标准化处理
x_train_vec = scaler.transform(x_train_vec)
x_test_vec = scaler.transform(x_test_vec)
#print y_train
model.fit(x_train_vec,y_train)
pp = model.predict(x_test_vec)

from sklearn.metrics import classification_report,confusion_matrix

# 展示一些统计结果 验证准确度
print(confusion_matrix(y_test,pp))
print(classification_report(y_test,pp))


score = model.score(x_test_vec, y_test)
print score

# 验证敏感性 
当换取4个类别后使用原model结果再测试一下准确度和可视化

In [ ]:
#train_model = Doc2Vec.load('docvec')
classDict = {
                      'soc.religion.christian':1,
                         'talk.politics.guns':2,
                         'talk.politics.mideast':3,
                          'talk.politics.misc':4,
                         
                           }
def choiced_data(params):
    x_train = []
    y_train = []
    for i,v in enumerate(params):  
        if v[1] in ['soc.religion.christian','talk.politics.guns','talk.politics.mideast','talk.politics.misc']:      
            temp = []
            temp.append(v[0])
            x_train.append(temp)
            y_train.append(classDict[v[1]])
    return x_train,y_train


#获取这四类新闻的文本和label
#x_train,y_train = helper.choiced_data(raw_data['20news-bydate-train'])
#x_test,y_test = helper.choiced_data(raw_data['20news-bydate-test'])

#将文本转化为文档向量
x_train_vec = []
x_test_vec = []
for i in x_train:
    #inferred_vector = train_model.infer_vector([token for token in gensim.utils.simple_preprocess(i[0]) if token not in STOPWORDS])
    inferred_vector = train_model.infer_vector(gensim.utils.simple_preprocess(i[0]))
    x_train_vec.append(inferred_vector)
for j in x_test:
    #inferred_vector_test = train_model.infer_vector([token for token in gensim.utils.simple_preprocess(j[0]) if token not in STOPWORDS])
    inferred_vector_test = train_model.infer_vector(gensim.utils.simple_preprocess(j[0]))
    x_test_vec.append(inferred_vector_test)

print x_train_vec[0],y_train[0]
print x_test_vec[0],y_test[0]

In [ ]:
from sklearn.manifold import TSNE
ts = TSNE(2)
reduced_vecs = ts.fit_transform(np.concatenate((x_test_vec[:376],x_test_vec[376:686],x_test_vec[686:1084],x_test_vec[1084:])))
print reduced_vecs.shape
for i in range(len(reduced_vecs)):
    if i < 376:
        color = 'b'
    elif i>=376 and i<686:
        color = 'r'
    elif i>=686 and i<1084:
        color = 'g'
    elif i>=1084:
        color = 'y'
    plt.plot(reduced_vecs[i,0],reduced_vecs[i,1],marker='o',color=color,markersize=3)
    
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

# 因为算法有设置自动停止 所以步长设置的比较小，迭代次数可以放的多
model =  MLPClassifier(hidden_layer_sizes=(50,25),max_iter=2000,random_state=0,verbose=True,alpha=1,learning_rate_init=0.0001)
scaler = StandardScaler()
scaler.fit(x_train_vec)

# 对得到的向量进行统一的标准化处理
x_train_vec = scaler.transform(x_train_vec)
x_test_vec = scaler.transform(x_test_vec)
#print y_train
model.fit(x_train_vec,y_train)
pp = model.predict(x_test_vec)

from sklearn.metrics import classification_report,confusion_matrix

# 展示一些统计结果 验证准确度
print(confusion_matrix(y_test,pp))
print(classification_report(y_test,pp))


score = model.score(x_test_vec, y_test)
print score
